# Дополнительный функционал (настройки)

In [1]:
runs = 10 # число запусков для подсчёта среднего времени
iterations = 75000 # сколько итераций должно пройти без улучшения ответа, чтобы алгоритм вернул текущий лучший

## Imports

In [2]:
import time # для подсчёта времени работы
import csv # для сохранения ответов
import pandas as pd # для вывода таблицы (v 2.1.1)
import random # для рандомизированного алгоритма
from tqdm import tqdm # для обёртки итераций между датасетами

import warnings # для скрития предупреждения о deprecated
warnings.simplefilter(action='ignore', category=FutureWarning)

## Вспомогательные функции (не участвуют в коде эвристик)

In [3]:
# вспомогательная функция для преобразования ответа (сортировка вершин клики в порядке возрастания)
def transform_answer(solution): # на вход solution вида (размер клики, [вершины в клике])
    for i in range(solution[0]):
        solution[1][i] += 1
    return [solution[0], sorted(solution[1])]

In [4]:
# # вспомогательная функция для проверки корректности полученного ответа
def check_solution(edges: dict, solution): # на вход словари edges с рёбрами и solution вида (размер клики, [вершины в клике])
    for i in range(solution[0]-1): # идём по вершинам и проверяем, смежны ли она со всеми остальными вершинами в найденной клике
        for j in range(i+1, solution[0]):
            if solution[1][j]-1 not in edges[solution[1][i]-1]: # проверяем наличие ребра (-1 чтобы работать с вершинами, так как их нумерация до вызова transform_answer шла с нуля)
                raise RuntimeError("Clique contains unconnected vertices!")

In [18]:
# вспомогательная функция для сохранения ответов в csv формате
def save_solution(solutions): # solutions - словарь всех полученных ответов
    for dataset in solutions.keys(): # идём по тест-кейсам
        with open(f'solutions/{dataset}.csv', 'w', newline='') as file:
            writer = csv.writer(file) # создаём объект для записи
            writer.writerow([solutions[dataset]["clique_size"]]) # сохраняем размер клики
            writer.writerows([solutions[dataset]["clique"]]) # сохраняем вершины клики
    
    # сохранение таблицы в csv формате
    table = pd.DataFrame(data = [], columns=["Instance", "Time, sec", "Clique size", "Clique vertices"])
    for dataset in solutions.keys(): # идём по тест-кейсам
        testcase = pd.DataFrame(data = [[dataset, solutions[dataset]["time"], solutions[dataset]["clique_size"], solutions[dataset]["clique"]]], columns=["Instance", "Time, sec", "Clique size", "Clique vertices"])
        table = pd.concat([table, testcase], ignore_index=True)
    table.to_csv("best_solution.csv", index=False)

In [6]:
# вспомогательная функция для вывода таблицы результатов
def show_results(solutions): # solutions - словарь всех полученных ответов
    table = pd.DataFrame(data = [], columns=["Instance", "Time, sec", "Clique size", "Clique vertices"])
    for dataset in solutions.keys(): # идём по тест-кейсам
        testcase = pd.DataFrame(data = [[dataset, solutions[dataset]["time"], solutions[dataset]["clique_size"], solutions[dataset]["clique"]]], columns=["Instance", "Time, sec", "Clique size", "Clique vertices"])
        table = pd.concat([table, testcase], ignore_index=True)
    display(table.style.hide()) # скрываем отображение индексов строк таблицы

# Получение данных

In [7]:
files = ["brock200_1", "brock200_2", "brock200_3", "brock200_4", "brock400_1", "brock400_2", "brock400_3", "brock400_4", "C125.9", "gen200_p0.9_44", "gen200_p0.9_55", "hamming8-4", "johnson16-2-4", "johnson8-2-4", "keller4", "MANN_a27", "MANN_a9", "p_hat1000-1", "p_hat1000-2", "p_hat1500-1", "p_hat300-3", "p_hat500-3", "san1000", "sanr200_0.9", "sanr400_0.7"] # файлы, на которых должен быть протестирован код

In [8]:
data = {} 
# data - словарь вида 
# {"название датасета" : 
#     {"vertex_num": число вершин, 
#     "edge_num": число рёбер, 
#     "edges": 
#         {словарь вида вершина - set смежных ей вершин}
#     }
#  ...
# }

In [9]:
for file in files:
    data[file] = {"vertex_num": None, "edge_num": None, "edges": {}}
    with open("data/" + file + ".clq", "r") as f: # открываем файл для чтения
        for row in f: # проходим по строкам
            if row[0] == "c": # если строка начинается с буквы "c" - это комментарий, пропускае строку
                continue
            elif row[0] == "p": # если строка начинается с буквы "p" - это описание проблемы, берём из этой строки число вершин и рёбер (последние два числа)
                data[file]["vertex_num"], data[file]["edge_num"] = int(row.split()[-2]), int(row.split()[-1])
            elif row[0] == "e": # если строка начинается с буквы "p" - это вершины, между которыми есть ребро
                v1, v2 = int(row.split()[-2]) - 1, int(row.split()[-1]) - 1 # запоминаем вершины (-1, чтобы не было мороки с индексацией)

                # добавляем связь вершины v1 с v2
                if v1 not in data[file]["edges"].keys(): # если это первое упоминание вершины v1 - создадим для неё set с указанием v2
                    data[file]["edges"][v1] = {v2}
                elif v2 not in data[file]["edges"][v1]: # иначе - просто добавим v2 в set смежных вершин v1
                    data[file]["edges"][v1].add(v2)

                # аналогично, но относительно вершины v2
                if v2 not in data[file]["edges"].keys():
                    data[file]["edges"][v2] = {v1}
                elif v1 not in data[file]["edges"][v2]:
                    data[file]["edges"][v2].add(v1)
        data[file]["edges"] = dict(sorted(data[file]["edges"].items())) # отсортируем вершины

In [10]:
data["johnson8-2-4"] # пример данных

{'vertex_num': 28,
 'edge_num': 210,
 'edges': {0: {5, 8, 9, 12, 13, 14, 17, 18, 19, 20, 23, 24, 25, 26, 27},
  1: {4, 7, 9, 11, 13, 14, 16, 18, 19, 20, 22, 24, 25, 26, 27},
  2: {3, 6, 9, 10, 13, 14, 15, 18, 19, 20, 21, 24, 25, 26, 27},
  3: {2, 7, 8, 11, 12, 14, 16, 17, 19, 20, 22, 23, 25, 26, 27},
  4: {1, 6, 8, 10, 12, 14, 15, 17, 19, 20, 21, 23, 25, 26, 27},
  5: {0, 6, 7, 10, 11, 14, 15, 16, 19, 20, 21, 22, 25, 26, 27},
  6: {2, 4, 5, 11, 12, 13, 16, 17, 18, 20, 22, 23, 24, 26, 27},
  7: {1, 3, 5, 10, 12, 13, 15, 17, 18, 20, 21, 23, 24, 26, 27},
  8: {0, 3, 4, 10, 11, 13, 15, 16, 18, 20, 21, 22, 24, 26, 27},
  9: {0, 1, 2, 10, 11, 12, 15, 16, 17, 20, 21, 22, 23, 26, 27},
  10: {2, 4, 5, 7, 8, 9, 16, 17, 18, 19, 22, 23, 24, 25, 27},
  11: {1, 3, 5, 6, 8, 9, 15, 17, 18, 19, 21, 23, 24, 25, 27},
  12: {0, 3, 4, 6, 7, 9, 15, 16, 18, 19, 21, 22, 24, 25, 27},
  13: {0, 1, 2, 6, 7, 8, 15, 16, 17, 19, 21, 22, 23, 25, 27},
  14: {0, 1, 2, 3, 4, 5, 15, 16, 17, 18, 21, 22, 23, 24, 27},
  15

# Реализация эвристики

### Greedy randomized maximum clique — простейшая реализация случайного (с учётом начальной степени вершин) алгоритма нахождения максимальной клики. В клику заносятся допустимые вершины (кандидаты) с вероятностями, пропорциональными изначальному количеству их соседей (то есть степени не пересчитываются при добавлении).

In [11]:
def randomized_greedy_max_clique(edges:dict, iterations=10):
    """
    Функция для получения начального рандомизированного решения задачи о максимальной клике.\n
    Parameters:
        * edges: словарь смежных вершинам вершин
        * iterations: через сколько попыток без улучшения решения выходить из алгоритма
    Returns:
        * tuple: (размер лучшей найденной клики, список вершин в этой клике)
    """
    # памятка - размер клики не может быть больше числа вешин

    original_candidates = set(edges.keys()) # set вершин (изначально все являются кандидатами в клику)
    original_candidates_degrees = [len(edges[v]) for v in original_candidates] # создаём список степеней вершин (индекс - номер вершины, так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)

    attempts = 0 # текущее число попыток
    best_clique = [] # текущая лучшая клика

    while attempts < iterations: # запускаем алгоритм, пока число попыток без изменения результата не превысит счётчик iterations
        clique = [] # создаём "пустую" клику
        candidates = original_candidates.copy() # копируем всех кандидатов
        while len(candidates) != 0: # пока есть кандидаты — пытаемся добавить их в клику
            candidates_degrees = [original_candidates_degrees[i] for i in candidates] # пересчитываем степени кандидатов (оставляем степени только рассматриваемых вершин) для итерациии случайного выбора
            
            v = random.choices(population=list(candidates), weights=candidates_degrees, k=1)[0] # случайным образом выбираем вершину в клику в соответствии с её степенью (чем больше степень относительно других вершин — тем выше вероятность) (переводим candidates в список для случайного выбора)
            clique.append(v) # добавляем её в клику

            candidates = candidates.intersection(edges[v]) # среди кандидитов оставляем только тех, кто смежен со всеми вершинами в текущей клике (итеративно этот список постоянно уменьшается с добавлением новых вершин в клику)

        if len(clique) > len(best_clique): # если нашли новую лучшую клику, то запоминаем её
            best_clique = clique.copy()
            attempts = 0 # обнуляем число итераций без улучшения решения
        else:
            attempts += 1 # увеличиваем число итераций без улучшения решения

    return len(best_clique), best_clique # возвращаем размер лучшей клики и её саму

In [ ]:
# тестирование времени работы 

# time_start = time.time() # замеряем время начала выполнения
# for i in range(5): # делаем 10 запусков для усреднения времени
#     sol = randomized_greedy_max_clique(data["brock200_4"]["edges"], iterations=100000)
# time_end = time.time() - time_start # считаем, сколько работал алгоритм
# print(time_end/5)

# print(Counter(random.choices(candidates, weights=candidates_degrees)[0]
#     for _ in range(100000)))

#### Тестирование

In [14]:
solutions_1 = {} 
# словарь для ответов первого варианта жадной эвристики вида
# {"название датасета" : 
#     {"clique_size": размер клики,
#      "clique": [вершины, входящие в клику],
#      "time": время на подсчёт
#     }
# }

for dataset in tqdm(data.keys()): # идём по тест-кейсам
    time_start = time.time() # замеряем время начала выполнения
    for i in range(runs): # делаем 10 запусков для усреднения времени
        sol = randomized_greedy_max_clique(data[dataset]["edges"], iterations=iterations)
    time_end = time.time() - time_start # считаем, сколько работал алгоритм
    # print("original solution", sol)
    sol = transform_answer(sol) # сортирует вершины клики в порядке возрастания их номера и возвращает нумерацию с единицы
    check_solution(data[dataset]["edges"], sol) # проверка решения
    # print("transformed solution", sol)
    solutions_1[dataset] = {"clique_size": sol[0], "clique": sol[1], "time": time_end/runs}

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [3:34:26<00:00, 514.66s/it]   


#### Вывод результатов

In [15]:
show_results(solutions_1)

Instance,"Time, sec",Clique size,Clique vertices
brock200_1,22.352904,21,"[18, 20, 39, 68, 73, 81, 85, 87, 90, 92, 93, 94, 102, 108, 134, 135, 136, 142, 150, 178, 186]"
brock200_2,7.868299,12,"[27, 48, 55, 70, 105, 120, 121, 135, 145, 149, 158, 183]"
brock200_3,10.703800,15,"[12, 29, 36, 38, 58, 84, 97, 98, 104, 118, 130, 144, 158, 173, 178]"
brock200_4,14.921686,17,"[12, 19, 28, 29, 38, 54, 65, 71, 79, 93, 117, 127, 139, 161, 165, 186, 192]"
brock400_1,38.695953,23,"[38, 46, 52, 62, 110, 122, 125, 128, 131, 141, 185, 192, 213, 218, 231, 247, 290, 295, 331, 333, 361, 368, 372]"
brock400_2,44.406454,23,"[35, 53, 69, 84, 86, 130, 143, 144, 153, 165, 168, 177, 205, 208, 219, 251, 261, 280, 296, 319, 326, 361, 377]"
brock400_3,45.493802,31,"[18, 20, 39, 68, 73, 85, 90, 92, 93, 102, 108, 134, 135, 142, 150, 178, 186, 207, 221, 223, 234, 252, 260, 262, 276, 304, 311, 348, 365, 380, 388]"
brock400_4,49.450677,23,"[26, 33, 97, 166, 180, 189, 192, 193, 226, 227, 234, 238, 247, 248, 255, 300, 314, 316, 317, 344, 376, 382, 385]"
C125.9,36.497941,34,"[1, 5, 7, 9, 11, 19, 25, 29, 31, 34, 44, 45, 49, 50, 52, 55, 65, 66, 68, 70, 77, 80, 85, 91, 96, 98, 99, 103, 104, 110, 114, 117, 122, 125]"
gen200_p0.9_44,68.102977,37,"[2, 13, 16, 20, 23, 27, 38, 51, 52, 58, 65, 72, 75, 86, 94, 96, 97, 100, 102, 108, 117, 120, 127, 138, 139, 141, 144, 146, 150, 151, 153, 166, 170, 175, 180, 186, 195]"


        Возможные улучшения: 
        1) Изначальный полностью жадный подсчёт клики (без элемента случайности) поможет сократить возможное число итераций и ускорит алгоритм.
        2) Для улучшения точности — после добавления вершины в клику можно пересчитывать степени смежных вершин (перестанут учитываться вершины, что не могут попасть в клику, при распределении весов в зависимости от ранга вершин). Однако этот подход может замедлить скорость работы алгоритма.
        3) Вместо полного случайного пересчёта решения — делать Destroy/Repair лучшего найденного.

# Вывод

In [19]:
save_solution(solutions_1) # сохранение лучших ответов